In [2]:
from gurobipy import *
import gurobipy as gp

# type declaration
from typing import Dict, List


import pandas as pd
import numpy as np

# %load_ext nb_black
# %load_ext lab_black

In [3]:
df_distances = pd.read_csv(
    "/home/angelo/Documents/Uni/Courses/Management Science/MM_Code_repo/data/distances.csv"
)
df_pairs = pd.read_csv(
    "/home/angelo/Documents/Uni/Courses/Management Science/MM_Code_repo/data/pairs.csv"
)

#### Data Prep

In [4]:
unique_c1 = list(df_distances.city1.unique())

# possible combinations
for c in unique_c1:
    d1 = [c, c, 0]
    df_distances = df_distances.append(
        pd.Series(d1, index=["city1", "city2", "distance"]), ignore_index=True
    )

/tmp/ipykernel_3861/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_distances = df_distances.append(
/tmp/ipykernel_3861/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_distances = df_distances.append(
/tmp/ipykernel_3861/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_distances = df_distances.append(
/tmp/ipykernel_3861/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_distances = df_distances.append(
/tmp/ipykernel_3861/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [5]:
df_pairs.loc[df_pairs.Dage == "-", "Dage"] = 99999999
df_pairs.Dage = df_pairs.Dage.astype(int)
df_pairs.loc[df_pairs.Rage == "-", "Rage"] = 99999999
df_pairs.Rage = df_pairs.Rage.astype(int)

df_pairs.loc[df_pairs.DBT == "-", "DBT"] = "NaN"
df_pairs.DBT = df_pairs.DBT.astype(str)
df_pairs.loc[df_pairs.RBT == "-", "RBT"] = "NaN"
df_pairs.RBT = df_pairs.RBT.astype(str)

In [6]:
# write the compatible bloodtypes
bloodtypes: Dict[str, int] = {
    # from A to everything
    ("A", "A"): 1,
    ("A", "B"): 0,
    ("A", "AB"): 1,
    ("A", "O"): 0,
    # from b to everything
    ("B", "A"): 0,
    ("B", "B"): 1,
    ("B", "AB"): 1,
    ("B", "O"): 0,
    # from AB to everything
    ("AB", "A"): 0,
    ("AB", "B"): 0,
    ("AB", "AB"): 1,
    ("AB", "O"): 0,
    # from O to everything
    ("O", "A"): 1,
    ("O", "B"): 1,
    ("O", "AB"): 1,
    ("O", "O"): 1,
    # missing blootypes
    ("NaN", "A"): 0,
    ("NaN", "B"): 0,
    ("NaN", "AB"): 0,
    ("NaN", "O"): 0,
    ("A", "NaN"): 0,
    ("B", "NaN"): 0,
    ("AB", "NaN"): 0,
    ("O", "NaN"): 0,
}

In [7]:
# change the order of the df to be more intuitive
RBT = df_pairs.loc[:, "RBT"]
DBT = df_pairs.loc[:, "DBT"]

# drop column
df_pairs.drop(columns=["RBT", "DBT"], axis=1, inplace=True)

# add them back
df_pairs["DBT"] = DBT
df_pairs["RBT"] = RBT

In [8]:
df_pairs["C"] = np.nan

In [9]:
for i, r in df_pairs.iterrows():
    if (
        (df_pairs.iloc[i].type == "pair")
        & (
            abs(df_pairs.iloc[i].Rage.astype(int) - df_pairs.iloc[i].Dage.astype(int))
            <= 10
        )
        & (bloodtypes[(df_pairs.iloc[i].DBT, df_pairs.iloc[i].RBT)] == 1)
    ) == True:
        df_pairs.loc[i, "C"] = 1
    else:
        df_pairs.loc[i, "C"] = 0

### create arcs

In [10]:
def pos_city_sublist(location_):
    """_summary_

    Args:
        location_ (_type_): _description_

    Returns:
        _type_: _description_
    """
    return list(
        df_distances[
            (df_distances.city1.astype(str) == location_)
            & (df_distances.distance <= 300)
        ].city2.values
    )


def blood_types_rbt(RBT_):
    """_summary_

    Args:
        RBT_ (_type_): _description_

    Returns:
        _type_: _description_
    """
    valid_bloodtypes_receiver_tmp = []
    for k, v in bloodtypes.items():
        if (k[1] == RBT_) & (v == 1):
            valid_bloodtypes_receiver_tmp.append(k[0])

    return valid_bloodtypes_receiver_tmp


def blood_types_dbt(DBT_):
    """_summary_

    Args:
        DBT_ (_type_): _description_

    Returns:
        _type_: _description_
    """
    valid_bloodtypes_donor_tmp = []
    for k, v in bloodtypes.items():
        if (k[0] == DBT_) & (v == 1):
            valid_bloodtypes_donor_tmp.append(k[1])

    return valid_bloodtypes_donor_tmp

In [11]:
arcs_: Dict[tuple, list] = {}

for i, r in df_pairs.iterrows():
    id_ = df_pairs.iloc[i].ID
    type_ = df_pairs.iloc[i].type
    Rage_ = df_pairs.iloc[i].Rage
    Dage_ = df_pairs.iloc[i].Dage
    location_ = df_pairs.iloc[i].Location
    compatible_ = df_pairs.iloc[i].C
    RBT_ = df_pairs.iloc[i].RBT
    DBT_ = df_pairs.iloc[i].DBT

    # pairs
    if type_ == "pair":

        # create a subset of cities that are valid
        possible_cities1 = pos_city_sublist(location_)

        # for receivers, create a donor vloodtype list that contains all the blood types that are compartible
        valid_bloodtypes_receiver = blood_types_rbt(RBT_)

        # same but inverted the donor blood types
        valid_bloodtypes_donor = blood_types_dbt(DBT_)

        arcs_[("pair_receiver", id_)] = list(
            df_pairs[
                (df_pairs.Location.isin(possible_cities1))
                & (abs(df_pairs.Dage - Rage_) <= 10)
                & (df_pairs.DBT.isin(valid_bloodtypes_receiver))
            ].ID.values
        )

        arcs_[("pair_donor", id_)] = list(
            df_pairs[
                (df_pairs.Location.isin(possible_cities1))
                & (abs(df_pairs.Rage - Dage_) <= 10)
                & (df_pairs.RBT.isin(valid_bloodtypes_donor))
            ].ID.values
        )

    elif type_ == "receiver":
        # grap the city and only select those rows in "city1" where  the df_distances.city.astye(str) == location_
        # create a subset of cities that are valid
        possible_cities1 = pos_city_sublist(location_)

        # for receivers, create a donor vloodtype list that contains all the blood types that are compartible
        valid_bloodtypes_receiver = blood_types_rbt(RBT_)

        arcs_[("receiver", id_)] = list(
            df_pairs[
                (df_pairs.Location.isin(possible_cities1))
                & (abs(df_pairs.Dage - Rage_) <= 10)
                & (df_pairs.DBT.isin(valid_bloodtypes_receiver))
            ].ID.values
        )

    elif type_ == "donor":
        # grap the city and only select those rows in "city1" where  the df_distances.city.astye(str) == location_
        # create a subset of cities that are valid
        possible_cities1 = pos_city_sublist(location_)

        # same but inverted the donor blood types
        valid_bloodtypes_donor = blood_types_dbt(DBT_)

        arcs_[("donor", id_)] = list(
            df_pairs[
                (df_pairs.Location.isin(possible_cities1))
                & (abs(df_pairs.Rage - Dage_) <= 10)
                & (df_pairs.RBT.isin(valid_bloodtypes_donor))
            ].ID.values
        )

    else:
        print("warning")
        break

In [12]:
arcs_l = []
for k, v in arcs_.items():
    for i in range(len(v)):
        if k[0] in ["donor", "pair_donor"]:
            arcs_l.append((k[1], v[i]))

In [13]:
len(arcs_l)

896

## Da model 

In [16]:
df_pairs['key'] = 0
# df2['key'] = 0

test = df_pairs.merge(df_pairs, on='key', how='outer')

In [19]:
test.drop(columns = "key", inplace = True)

In [25]:
test.drop(test[test.type_x == "receiver"].index, axis = 0, inplace = True)

In [28]:
df_distances

,city1,city2,distance
0,Amsterdam,London,222
1,London,Frankfurt,397
2,London,Amsterdam,222
3,Paris,Amsterdam,267
4,Frankfurt,Paris,297
...,...,...,...
319,Manchester,Manchester,0
320,Milan,Milan,0
321,Palma de Mallorca,Palma de Mallorca,0
322,Lisbon,Lisbon,0


In [47]:
test['distances_km'] = np.nan
test.reset_index(inplace = True, drop = True)
for i, r in test.iterrows():
    loc_city1 = r.Location_x
    loc_city2 = r.Location_y

    distance = df_distances[(df_distances.city1 == loc_city1) & (df_distances.city2 == loc_city2)].distance.values[0]
    test.iloc[i,-1] = distance

0
Paris
1
Moscow
2
Istanbul
3
Frankfurt
4
Istanbul
5
Madrid
6
Istanbul
7
Vienna
8
Rome
9
Milan
10
Rome
11
Madrid
12
Palma de Mallorca
13
Rome
14
Rome
15
Moscow
16
Palma de Mallorca
17
Moscow
18
Moscow
19
Düsseldorf
20
Moscow
21
Vienna
22
Oslo
23
Amsterdam
24
Palma de Mallorca
25
Moscow
26
Vienna
27
Oslo
28
Lisbon
29
Amsterdam
30
Paris
31
Oslo
32
Zürich
33
Copenhagen
34
Milan
35
Amsterdam
36
Milan
37
Manchester
38
Moscow
39
Istanbul
40
Vienna
41
London
42
Madrid
43
London
44
Zürich
45
London
46
Paris
47
Frankfurt
48
Düsseldorf
49
Lisbon
50
Istanbul
51
Oslo
52
Milan
53
Istanbul
54
Dublin
55
Istanbul
56
Palma de Mallorca
57
Zürich
58
Frankfurt
59
London
60
Lisbon
61
Vienna
62
Amsterdam
63
London
64
Lisbon
65
Palma de Mallorca
66
Düsseldorf
67
Copenhagen
68
Düsseldorf
69
Vienna
70
Düsseldorf
71
Vienna
72
Palma de Mallorca
73
Madrid
74
Rome
75
Amsterdam
76
Madrid
77
Madrid
78
Amsterdam
79
Copenhagen
80
Manchester
81
Dublin
82
Frankfurt
83
London
84
Moscow
85
Milan
86
Manchester
87
Zürich
88

In [15]:
# list of donors & pairs
d_p = list(df_pairs[df_pairs.type.isin(["donor", "pair"])].ID.values)

# list of receivers and pairs
r_p = list(df_pairs[df_pairs.type.isin(["receiver", "pair"])].ID.values)

In [16]:
donor_receiver_combos = []

for donor in d_p:
    for receiver in r_p:
        donor_receiver_combos.append((donor, receiver))

In [17]:
arcs_l_dict = {}
for paring in donor_receiver_combos:
    if paring in arcs_l:
        arcs_l_dict[paring] = 1
    else:
        arcs_l_dict[paring] = 0

In [20]:
model = gp.Model()

x = model.addVars(d_p, r_p, obj=1, vtype=GRB.BINARY, name="x")
model.ModelSense = gp.GRB.MAXIMIZE

model.setParam('TimeLimit', 10)




# for donor_or_pair_id in d_p:
#     lhs = 0
#     for pairing in arcs_l:
#         if pairing[0] == donor_or_pair_id:
#             lhs += x[pairing]
#     model.addConstr(lhs <= 1)





for donor_or_pair_id in range(201):
    if donor_or_pair_id in d_p:
        lhs = 0
        for pairing in arcs_l:
            if pairing[0] == donor_or_pair_id:
                lhs += x[pairing]
        model.addConstr(lhs <= 1)
    else:
        lhs = 0
        for pairing in arcs_l:
            if pairing[0] == donor_or_pair_id:
                lhs += x[pairing]
        model.addConstr(lhs == 0)





# model.optimize()
# model.printAttr("X")
# print("Kidney Transplants", model.ObjVal)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Set parameter TimeLimit to value 10


In [21]:
model.write("problem.lp")
with open("problem.lp") as f:
    print(f.read())

\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  x[2,1] + x[2,2] + x[2,3] + x[2,4] + x[2,5] + x[2,6] + x[2,7] + x[2,8]
   + x[2,9] + x[2,10] + x[2,11] + x[2,12] + x[2,15] + x[2,16] + x[2,18]
   + x[2,19] + x[2,20] + x[2,21] + x[2,22] + x[2,23] + x[2,24] + x[2,25]
   + x[2,26] + x[2,27] + x[2,28] + x[2,29] + x[2,30] + x[2,32] + x[2,33]
   + x[2,34] + x[2,35] + x[2,37] + x[2,38] + x[2,39] + x[2,40] + x[2,42]
   + x[2,43] + x[2,44] + x[2,45] + x[2,46] + x[2,47] + x[2,48] + x[2,49]
   + x[2,50] + x[2,51] + x[2,52] + x[2,53] + x[2,54] + x[2,56] + x[2,57]
   + x[2,58] + x[2,59] + x[2,60] + x[2,62] + x[2,65] + x[2,66] + x[2,69]
   + x[2,70] + x[2,71] + x[2,72] + x[2,73] + x[2,74] + x[2,75] + x[2,76]
   + x[2,77] + x[2,78] + x[2,79] + x[2,80] + x[2,81] + x[2,82] + x[2,83]
   + x[2,84] + x[2,85] + x[2,86] + x[2,87] + x[2,88] + x[2,89] + x[2,90]
   + x[2,91] + x[2,92] + x[2,93] + x[2,94] + x[2,96] + x[2,97] + x[2,98]
   + x[2,99] + x[2,100] + x[2,101] + x